# 02 - Exploratory Data Analysis (EDA)

This notebook focuses on analyzing the dataset through descriptive statistics and visualizations.
The key steps include:
- Understanding data distribution
- Identifying correlations
- Exploring trends in loan defaults

## 1. Data Overview


In [156]:
keep_list = ['charged_off','funded_amnt','addr_state', 'annual_inc', 'application_type', 
             'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 
             'fico_range_low', 'grade', 'home_ownership', 'id', 'initial_list_status', 
             'installment', 'int_rate', 'loan_amnt', 'loan_status', 'mort_acc', 'open_acc', 
             'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 
             'sub_grade', 'term', 'title', 'total_acc', 'verification_status', 'zip_code',
             'last_pymnt_amnt','num_actv_rev_tl', 'mo_sin_rcnt_rev_tl_op','mo_sin_old_rev_tl_op',
             "bc_util","bc_open_to_buy","avg_cur_bal","acc_open_past_24mths" ]

In [157]:
filtered_df = filtered_df[filtered_df.columns.intersection(keep_list)]

In [159]:
# Set the labels to be 1 for "Charged Off" and 0 for "Fully Paid"
filtered_df['target'] = filtered_df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)
filtered_df.drop('loan_status', axis=1, inplace=True)

In [160]:
correlation_with_target = filtered_df.corr()['target'].sort_values(ascending=False).abs()
bad_correlation_with_target = correlation_with_target[correlation_with_target<0.03]
filtered_df.drop(bad_correlation_with_target.index, axis=1, inplace=True)

## 2. Data Visualization
- Distribution of key numerical features
- Categorical feature analysis
- Loan default trends

In [165]:
processed_df = filtered_df.copy()

In [166]:
# Select non-numeric features
non_numeric_features = filtered_df.select_dtypes(include=['object']).columns

# Display the number of categories for each non-numeric feature
for feature in non_numeric_features:
    print(f"{feature}: {filtered_df[feature].nunique()} categories")

term: 2 categories
grade: 7 categories
sub_grade: 35 categories
emp_title: 32987 categories
emp_length: 11 categories
home_ownership: 4 categories
verification_status: 3 categories
purpose: 13 categories
title: 20 categories
zip_code: 846 categories
addr_state: 46 categories
earliest_cr_line: 598 categories
initial_list_status: 2 categories
application_type: 1 categories


In [167]:
processed_df['term'] = filtered_df['term'].copy().apply(lambda row: 36 if row == ' 36 months' else 60)

## 3. Correlation Analysis
- Checking relationships between variables
- Identifying important predictors for loan defaults

In [168]:
# Define base values for each letter, where A is the highest and G is the lowest
letter_values = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}

# Convert grades to a continuous numeric scale
def grade_to_continuous(grade):
    letter = grade[0]  # Extract the letter part
    number = int(grade[1])  # Extract the number part
    return letter_values[letter] * 5 - number  # Use a continuous scoring system

# Apply the function to the 'grade' column
processed_df['sub_grade'] = filtered_df['sub_grade'].copy().apply(grade_to_continuous)

# We can drop grade because we have sub_grade
processed_df.drop('grade', axis=1, inplace=True)
# processed_df[['sub_grade']]

In [169]:
experience_mapping = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 12
}   

# Apply the function to the 'grade' column
processed_df['emp_length'] = filtered_df['emp_length'].copy().map(experience_mapping)

In [170]:
processed_df['initial_list_status'] = np.where(filtered_df['initial_list_status'] == 'w', 0, 1)